# Fund Volume Data

Based on models: Fund, Issuer, FundVolumeData, Monthly

To understand more about the models and its data, you can go to Django admin site.

Connecting the Django project to iPython/Jupyter notebook.

In [1]:
import os
import sys
import django
sys.path.append("../")
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_pandas_examples.settings'
django.setup()

In [2]:
from core.models import FundVolumeData, Fund, Issuer, Monthly
from django_pandas.io import read_frame
import pandas as pd

In [3]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [4]:
read_frame(Issuer.objects.all(), fieldnames=['symbol', 'name'], verbose=False)

,symbol,name
0,TD,TD Greenline
1,AIC,AIC Financial Group Ltd
2,ANSA,Ansa Merchant Bank Limited
3,BOURS,Bourse Securities Limited
4,CLICO,Colonial Life Insurance Company (Trinidad) Lim...
5,FRTEM,Franklin Templeton
6,FCB,First Citizens Asset Management Limited
7,GAM,Guardian Asset Management Limited
8,JMMB,JMMB Securities Limited
9,RBTT,RBTT Trust and Asset Management Limited


In [5]:
funds = read_frame(Fund.objects.all().order_by('issuer__name').filter(active=True), 
             fieldnames=['issuer__name', 'issuer__symbol', 
                         'description', 'symbol'], 
             verbose=False)

In [6]:
funds

,issuer__name,issuer__symbol,description,symbol
0,AIC Financial Group Ltd,AIC,AIC TT Short Term Income Fund,AICSTI
1,AIC Financial Group Ltd,AIC,AIC TT (US$) INCOME FUND,AICUSI
2,AIC Financial Group Ltd,AIC,AIC TT Income Fund,AICTTI
3,AIC Financial Group Ltd,AIC,AIC TT Income and Growth Fund,AICIG
4,AIC Financial Group Ltd,AIC,AIC TT Caribbean Equity Fund,AICCE
5,AIC Funds Inc.,AICFI,AIC American Advantage Corporate Class,AICAACC
6,AIC Funds Inc.,AICFI,AIC American Focused Corporate Class,AICAFCC
7,AIC Funds Inc.,AICFI,AIC Global Advantage Corporate Class,AICGACC
8,AIC Funds Inc.,AICFI,AIC Global Premium Dividend Income Corporate C...,AICGDCC
9,AIC Funds Inc.,AICFI,AIC Value Corporate Class,AICVCC


In [7]:
qs = FundVolumeData.objects.values_list(
                                    'fund__issuer__symbol', 'fund__symbol', 'period_ending__dateix', 
                                   'total_net_assets_under_management', 'total_unit_holders').filter(
                                    fund__issuer__symbol='UTC',period_ending__dateix__gt='2009-12-31').order_by(
                                    'period_ending__dateix')

In [8]:
fields = [
    'period_ending__dateix', 
    'fund__issuer__symbol', 
    'fund__symbol', 
    'total_net_assets_under_management', 
    'total_unit_holders'
]

pivot_columns = [
    'fund__issuer__symbol', 
    'fund__symbol'
]

In [9]:
ts = qs.to_timeseries(fields,
                     index='period_ending__dateix', pivot_columns=pivot_columns, 
                     values='total_net_assets_under_management',
                     storage='long',
                     verbose=False
                     )

In [10]:
ts

combined_keys,UTC.UTCAP.total_net_assets_under_management,UTC.UTCCIG.total_net_assets_under_management,UTC.UTCE.total_net_assets_under_management,UTC.UTCEU.total_net_assets_under_management,UTC.UTCFUS.total_net_assets_under_management,UTC.UTCGB.total_net_assets_under_management,UTC.UTCLA.total_net_assets_under_management,UTC.UTCSUS.total_net_assets_under_management,UTC.UTCUSMM.total_net_assets_under_management,UTC.UTCUVR.total_net_assets_under_management
period_ending__dateix,,,,,,,,,,
2010-01-31,9330082.12444,2.083312e+08,29059171.6206,6125149,3182932635,10042182.1640,6459684.28536,10409307990,4.991258e+09,1.501717e+08
2010-02-28,10630510.98260,2.132580e+08,29244557.8583,6125584,3212810632,10017028.5718,6676341.19734,10556573261,4.955957e+09,1.513360e+08
